In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris

from poliastro.bodies import Sun,Venus, Earth, Jupiter, Saturn
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.iod import izzo
from poliastro.plotting import OrbitPlotter2D
from poliastro.util import norm
import poliastro.twobody.propagation as Propagation

from scipy import optimize as opt
 
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [2]:
cassini_venus2 = Time("1999-06-24")
cassini_earth2 = Time("1999-08-18")
cassini_jupiter = Time("2000-12-30")
cassini_saturn = Time("2004-07-01")

#cassini transit
cassini_js_time = cassini_saturn-cassini_jupiter
cassini_ej_time = cassini_jupiter-cassini_earth2
cassini_ve_time = cassini_earth2-cassini_venus2

In [3]:
tmod=[0,0,0,0]
plot_on=0;

def trajectory_calculator(t=[0,0,0,0],plot_on=0,disp_on=0):
    #start with cassini style assumptions
    cassini_venus2 = Time("1999-06-24")
    cassini_earth2 = Time("1999-08-18")
    cassini_jupiter = Time("2000-12-30")
    cassini_saturn = Time("2004-07-01")

    #cassini transit
    cassini_js_time = cassini_saturn-cassini_jupiter
    cassini_ej_time = cassini_jupiter-cassini_earth2
    cassini_ve_time = cassini_earth2-cassini_venus2


    tmod_s  = t[0] #arrival time in saturn relative to 2020
    tmod_js = t[1] #transit times
    tmod_ej = t[2] 
    tmod_ve = t[3]


    time_js = tmod_js*u.year
    time_ej = tmod_ej*u.year
    time_ve = tmod_ve*u.year

    date_arrive_saturn = Time("2020-01-01", scale="utc")+tmod_s*u.year;


    #create target orbit
    o_sf = Orbit.from_body_ephem(Saturn, date_arrive_saturn)
    r_sf, v_sf = o_sf.rv()
    #o_sf.plot()
    if disp_on:
        print('computed saturn')

    ################## Jupiter ################## 

    #guess flyby date of Jupiter
    date_flyby_jupiter = date_arrive_saturn-time_js

    #construct j orbit
    o_j = Orbit.from_body_ephem(Jupiter, date_flyby_jupiter)
    #o_j.plot()

    #compute transfer lambert trajectory
    (v_jo, v_si), = izzo.lambert(Sun.k, o_j.r, o_sf.r, time_js)
    trx_js = Orbit.from_vectors(Sun, o_j.r, v_jo, epoch=date_flyby_jupiter)
    #trx_js.plot()
    if disp_on:
        print('computed J-S')
    ################## Earth ################## 
    #guess flyby date of Earth
    date_flyby_earth = date_arrive_saturn-time_js-time_ej

    #construct j orbit
    o_e2 = Orbit.from_body_ephem(Earth, date_flyby_earth)
    #o_e2.plot()

    #compute transfer lambert trajectory
    (v_eo, v_ji), = izzo.lambert(Sun.k, o_e2.r, o_j.r, time_ej)
    trx_ej = Orbit.from_vectors(Sun, o_e2.r, v_eo, epoch=date_flyby_earth)
    #trx_ej.plot()
    if disp_on:
        print('computed E-J')                            

    ################## Venus2 ################## 
    #guess flyby date of Venus2
    date_flyby_venus2 = date_arrive_saturn-time_js-time_ej-time_ve

    #construct j orbit
    o_v2 = Orbit.from_body_ephem(Venus, date_flyby_venus2)
    #o_v2.plot()

    #compute transfer lambert trajectory
    (v_v2o, v_ei), = izzo.lambert(Sun.k, o_v2.r, o_e2.r, time_ve)
    trx_v2e = Orbit.from_vectors(Sun, o_v2.r, v_v2o, epoch=date_flyby_venus2)
    #trx_v2e.plot()    
    if disp_on:
        print('computed V-E')                            

    ################## Sum delta v ##################                             
    delv_e = norm(v_eo-v_ei)
    delv_j = norm(v_jo-v_ji)
    delv_s = norm(v_si-v_sf)

    total_deltav=sum([delv_e,delv_j,delv_s]) 

    if disp_on:
        print('Total delta-v: ', total_deltav)

    ################## Plot ##################  

    if plot_on:
        op = OrbitPlotter2D()

        op.plot(o_v2,label="Venus2 Orbit")
        op.plot(o_e2,label="Earth2 Orbit")
        op.plot(o_j, label="Jupiter Orbit")
        op.plot(o_sf, label="Saturn Orbit")


        op.plot(trx_v2e, label="V2-E")
        op.plot(trx_ej, label="E-J")
        op.plot(trx_js, label="J-S")
        
    orbits = (o_v2,o_e2,o_j,o_sf)
    trajectories = (trx_v2e,trx_ej,trx_js)
    deltavs = (delv_e,delv_j,delv_s)
    times = (date_flyby_venus2,date_flyby_earth,date_flyby_jupiter,date_arrive_saturn)
    return (total_deltav,orbits,trajectories,deltavs,times)

def print_times(times):
    print("Venus:   ",times[0])
    print("Earth:   ",times[1])
    print("Jupiter: ",times[2])
    print("Saturn:  ",times[3])
    print("**")
    print("V-E: ", (times[1]-times[0]).to(u.year))
    print("E-J: ", (times[2]-times[1]).to(u.year))
    print("E-S: ", (times[3]-times[2]).to(u.year))
        



In [4]:
cassini_js_time.to(u.year).value

3.5017111567419574

In [5]:
t_guess_cassini = [27,cassini_js_time.to(u.year).value,cassini_ej_time.to(u.year).value,cassini_ve_time.to(u.year).value]
sol=trajectory_calculator(t=t_guess_cassini,plot_on=0,disp_on=1)

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2046-12-31 18:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-07-01 18:01:09.184. Use Time(..., scale='tdb') instead.



computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  82.09792665510716 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-02-16 18:01:09.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-12-23 18:01:09.184. Use Time(..., scale='tdb') instead.



In [ ]:
sol=trajectory_calculator(tmod=[0,0,0,0],plot_on=0,disp_on=1)
if 1:
    op=OrbitPlotter2D()
    op.plot(sol[1][0],label='V')
    op.plot(sol[1][1],label='E')
    op.plot(sol[1][2],label='J')
    op.plot(sol[1][3],label='S')
    op.plot(sol[2][0],label='VE')
    op.plot(sol[2][1],label='EJ')
    op.plot(sol[2][2],label='JS')



/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-03-29 00:01:09.186. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-09-27 00:01:09.182. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-05-15 00:01:09.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-03-21 00:01:09.186. Use Time(..., scale='tdb') instead.



computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  49.73689056630519 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



In [6]:
sol

(<Quantity 82.09792665510716 km / s>,
 (1 x 1 AU x 24.4 deg (ICRS) orbit around Sun (☉) at epoch 2041-12-23 18:01:09.184 (TDB),
  1 x 1 AU x 23.4 deg (ICRS) orbit around Sun (☉) at epoch 2042-02-16 18:01:09.185 (TDB),
  5 x 5 AU x 23.2 deg (ICRS) orbit around Sun (☉) at epoch 2043-07-01 18:01:09.184 (TDB),
  9 x 10 AU x 22.6 deg (ICRS) orbit around Sun (☉) at epoch 2046-12-31 18:01:09.184 (TDB)),
 (30293036 x -2883088647 km x 24.0 deg (HCRS) orbit around Sun (☉) at epoch 2041-12-23 18:00:00.000 (UTC),
  1 x 10 AU x 23.2 deg (HCRS) orbit around Sun (☉) at epoch 2042-02-16 18:00:00.000 (UTC),
  0 x 47 AU x 27.1 deg (HCRS) orbit around Sun (☉) at epoch 2043-07-01 18:00:00.000 (UTC)),
 (<Quantity 38.076032380696105 km / s>,
  <Quantity 29.27965312341461 km / s>,
  <Quantity 14.742241150996456 km / s>),
 (<Time object: scale='utc' format='iso' value=2041-12-23 18:00:00.000>,
  <Time object: scale='utc' format='iso' value=2042-02-16 18:00:00.000>,
  <Time object: scale='utc' format='iso' val

In [7]:
t_guess_cassini

[27, 3.5017111567419574, 1.3689253935660506, 0.15058179329226556]

In [15]:
t_guess_cassini = [18,cassini_js_time.to(u.year).value,cassini_ej_time.to(u.year).value,cassini_ve_time.to(u.year).value]
bounds = ((10,20),(0.01,5),(0.01,3),(0.01,1))
options = {'maxiter': 30,'disp': True}

def total_deltav_calculator(tmod):
    print(tmod)
    sol=trajectory_calculator(tmod,plot_on=0,disp_on=0)
    print(sol[0])
    return sol[0].to(u.km/u.s).value

opt_sol = opt.minimize(total_deltav_calculator,x0=t_guess_cassini,options=options,bounds=bounds)

opt_sol

[ 18.           3.50171116   1.36892539   0.15058179]
79.18253603560625 km / s
[ 18.00000001   3.50171116   1.36892539   0.15058179]
79.18253455682253 km / s
[ 18.           3.50171117   1.36892539   0.15058179]
79.18253741852827 km / s
[ 18.           3.50171116   1.3689254    0.15058179]
79.18253723952036 km / s
[ 18.           3.50171116   1.36892539   0.1505818 ]
79.18253497196739 km / s
[  2.00000000e+01   1.00000000e-02   1.00000000e-02   1.00000000e+00]
5616.589019747007 km / s
[  2.00000000e+01   1.00000000e-02   1.00000000e-02   1.00000000e+00]
5616.588964607887 km / s
[  2.00000000e+01   1.00000100e-02   1.00000000e-02   1.00000000e+00]
5616.587345878905 km / s
[  2.00000000e+01   1.00000000e-02   1.00000100e-02   1.00000000e+00]
5616.585161113126 km / s

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2037-12-31 12:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2034-07-01 12:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2033-02-16 12:01:09.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2032-12-23 12:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 


[  2.00000000e+01   1.00000000e-02   1.00000000e-02   1.00000001e+00]
5616.589019167613 km / s
[ 18.74169365   2.20682116   0.86497223   0.46558584]
90.91188196574512 km / s
[ 18.74169366   2.20682116   0.86497223   0.46558584]
90.91188313406796 km / s
[ 18.74169365   2.20682117   0.86497223   0.46558584]
90.9118806279785 km / s
[ 18.74169365   2.20682116   0.86497224   0.46558584]
90.91188033992796 km / s
[ 18.74169365   2.20682116   0.86497223   0.46558585]
90.91188105110047 km / s
[ 18.33419339   2.91825776   1.14185345   0.29251677]
77.83777545685919 km / s
[ 18.3341934    2.91825776   1.14185345   0.29251677]
77.8377752079213 km / s
[ 18.33419339   2.91825777   1.14185345   0.29251677]
77.83777558198452 km / s
[ 18.33419339   2.91825776   1.14185346   0.29251677]
77.83777528508602 km / s
[ 18.33419339   2.91825776   1.14185345   0.29251678]
77.83777416774129 km / s
[ 18.47631719   2.70469692   1.17390917   0.73221881]


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2038-09-28 09:42:20.630. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2036-07-14 08:42:41.035. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2035-09-02 10:21:53.461. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2035-03-16 09:02:21.846. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

81.74550994042472 km / s
[ 18.4763172    2.70469692   1.17390917   0.73221881]
81.74551109843445 km / s
[ 18.47631719   2.70469693   1.17390917   0.73221881]
81.7455086454502 km / s
[ 18.47631719   2.70469692   1.17390918   0.73221881]
81.74550853962631 km / s
[ 18.47631719   2.70469692   1.17390917   0.73221882]
81.7455092174843 km / s
[ 18.37889723   2.851084     1.15193631   0.43082133]
76.47339745525795 km / s
[ 18.37889724   2.851084     1.15193631   0.43082133]
76.4733956713364 km / s
[ 18.37889723   2.85108401   1.15193631   0.43082133]
76.47339911186768 km / s
[ 18.37889723   2.851084     1.15193632   0.43082133]


...

27.068558324917362 km / s
[ 20.           3.4290545    1.90108617   0.16826689]
27.068558323721714 km / s
[ 20.           3.4290545    1.90108616   0.1682669 ]
27.068557606501543 km / s
[ 20.           3.45284209   1.87897398   0.1721752 ]
26.944396059658075 km / s
[ 20.00000001   3.45284209   1.87897398   0.1721752 ]
26.944395995007575 km / s
[ 20.           3.4528421    1.87897398   0.1721752 ]
26.944396050629628 km / s
[ 20.           3.45284209   1.87897399   0.1721752 ]
26.944396045432896 km / s
[ 20.           3.45284209   1.87897398   0.17217521]
26.94439596393055 km / s
[ 20.           3.40492324   1.92553238   0.17275405]
26.929084740338002 km / s
[ 20.00000001   3.40492324   1.92553238   0.17275405]
26.92908471506907 km / s
[ 20.           3.40492325   1.92553238   0.17275405]
26.929084690525244 km / s
[ 20.           3.40492324   1.92553239   0.17275405]
26.929084688425316 km / s
[ 20.           3.40492324   1.92553238   0.17275406]
26.929084755527878 km / s
[ 20.           

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2036-08-05 08:27:44.007. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2034-09-02 01:14:43.395. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2034-06-30 22:53:00.053. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2036-08-05 08:27:43.376. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

26.92837427492946 km / s
[ 20.           3.39882233   1.93161321   0.17276811]
26.92837427443158 km / s
[ 20.           3.39882233   1.9316132    0.17276812]
26.928374295555173 km / s
[ 20.           3.39830699   1.9321789    0.17276969]
26.928322089384356 km / s
[ 20.00000001   3.39830699   1.9321789    0.17276969]
26.92832201447039 km / s
[ 20.           3.398307     1.9321789    0.17276969]
26.92832208901571 km / s
[ 20.           3.39830699   1.93217891   0.17276969]
26.928322089014348 km / s
[ 20.           3.39830699   1.9321789    0.1727697 ]
26.928322089560293 km / s
[ 20.           3.39842047   1.93207053   0.17276996]
26.928322000845817 km / s
[ 20.00000001   3.39842047   1.93207053   0.17276996]
26.928321925575077 km / s
[ 20.           3.39842048   1.93207053   0.17276996]
26.92832200083983 km / s
[ 20.           3.39842047   1.93207054   0.17276996]
26.928322000840307 km / s
[ 20.           3.39842047   1.93207053   0.17276997]
26.92832200085186 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2036-08-07 17:27:54.953. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2034-09-02 00:56:06.130. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2034-06-30 22:26:00.760. Use Time(..., scale='tdb') instead.



      fun: 26.928322000845817
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -7.52707408e+00,  -5.98632255e-04,  -5.51025892e-04,
         6.04316597e-04])
  message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 260
      nit: 30
   status: 1
  success: False
        x: array([ 20.        ,   3.39842047,   1.93207053,   0.17276996])

In [27]:
t_guess_cassini

[27, 3.5017111567419574, 1.3689253935660506, 0.15058179329226556]

In [ ]:

options = {'maxiter': 30,'disp': False}
opt_sol = opt.minimize(total_deltav_calculator,x0=opt_sol.x,options=options)
opt_sol

In [16]:
opt_sol

      fun: 26.928322000845817
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -7.52707408e+00,  -5.98632255e-04,  -5.51025892e-04,
         6.04316597e-04])
  message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 260
      nit: 30
   status: 1
  success: False
        x: array([ 20.        ,   3.39842047,   1.93207053,   0.17276996])

In [8]:
sol=trajectory_calculator(t=opt_sol.x,plot_on=0,disp_on=0)

print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]


trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_ej/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')

NameError: name 'opt_sol' is not defined

In [14]:
norm(sol[1][2].v).to(u.km/u.s)

<Quantity 13.433691807592876 km / s>

In [6]:
#tmod = opt_sol.x
tmod = [27.238877,4.798,1.2879,0.1313]
sol=trajectory_calculator(t=tmod,plot_on=0,disp_on=0)

#print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]


trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_ej/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-03-29 00:00:54.001. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-06-10 12:44:49.200. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-02-25 03:00:56.161. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-01-08 04:02:23.279. Use Time(..., scale='tdb') instead.




Total deltav:  6.010478813049802 km / s
Delta vs:  [0.19207023597361006, 0.0776495284180622, 5.74075904865813]

Times:
Venus:    2041-01-08 04:01:14.095
Earth:    2041-02-25 02:59:46.975
Jupiter:  2042-06-10 12:43:40.015
Saturn:   2047-03-28 23:59:44.815
**
V-E:  0.1313 yr
E-J:  1.2879 yr
E-S:  4.798 yr



/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [42]:
trx_ve.p.to(u.AU)

<Quantity 1.3025304891234142 AU>

In [45]:
norm(trx_ve.r).to(u.AU)

<Quantity 0.7250507024742522 AU>

In [32]:
trx_ve.p

<Quantity 194855787.69469228 km>

In [33]:
trx_ve.ecc

<Quantity 0.8651263559885476>

In [34]:
trx_ve.inc.to(u.deg)

<Quantity 22.826994822375408 deg>

In [35]:
trx_ve.raan.to(u.deg)

<Quantity 0.6651624083357963 deg>

In [36]:
trx_ve.argp.to(u.deg)

<Quantity 87.40383111130444 deg>

In [37]:
trx_ve.nu.to(u.deg)

<Quantity -22.980477233716012 deg>

In [46]:
trx_ve.r

<Quantity [ 45776760.73223779, 90712263.57357913, 37955968.12664004] km>

In [47]:
trx_ve.v

<Quantity [-46.2230461 , 10.7910711 ,  4.76769139] km / s>

In [ ]:
trx_ve_half = trx_ve.propagate(t_ve/2)

In [ ]:
trx_ve_half.plot()

In [ ]:
norm(o_e.v.to(u.km/u.s))

In [ ]:
print_times(sol[4])

In [48]:
sol

(<Quantity 6.010478813049802 km / s>,
 (1 x 1 AU x 24.5 deg (ICRS) orbit around Sun (☉) at epoch 2041-01-08 04:02 (TDB),
  1 x 1 AU x 23.4 deg (ICRS) orbit around Sun (☉) at epoch 2041-02-25 03:00 (TDB),
  5 x 5 AU x 23.2 deg (ICRS) orbit around Sun (☉) at epoch 2042-06-10 12:44 (TDB),
  9 x 10 AU x 22.6 deg (ICRS) orbit around Sun (☉) at epoch 2047-03-29 00:00 (TDB)),
 (1 x 10 AU x 22.8 deg (HCRS) orbit around Sun (☉) at epoch 2041-01-08 04:01:14.095 (UTC),
  1 x 10 AU x 22.6 deg (HCRS) orbit around Sun (☉) at epoch 2041-02-25 02:59:46.975 (UTC),
  1 x 10 AU x 22.4 deg (HCRS) orbit around Sun (☉) at epoch 2042-06-10 12:43:40.015 (UTC)),
 (<Quantity 0.19207023597361006 km / s>,
  <Quantity 0.0776495284180622 km / s>,
  <Quantity 5.74075904865813 km / s>),
 (<Time object: scale='utc' format='iso' value=2041-01-08 04:01:14.095>,
  <Time object: scale='utc' format='iso' value=2041-02-25 02:59:46.975>,
  <Time object: scale='utc' format='iso' value=2042-06-10 12:43:40.015>,
  <Time object:

In [34]:
(Time("2047-03-29", scale="utc")-Time("2020-01-01", scale="utc")).to(u.year)

<Quantity 27.238877481177276 yr>

In [52]:
(norm(sol[1][3].v).to(u.km/u.s))**2

<Quantity 83.6607810811809 km2 / s2>

In [27]:
norm(sol[1][1].v).to(u.km/u.s)

<Quantity 29.940786635191444 km / s>

In [85]:
Earth.R

<<class 'astropy.constants.constant.Constant'> name='Earth equatorial radius' value=6378136.6 uncertainty=0.0001 unit='m' reference='IAU Working Group on Cartographic Coordinates and Rotational Elements: 2009'>

In [90]:
earthflybyConstraint(sol)

35.717495055476995 km / s
29.94078663519144 km / s
[ -0.40886024  36.70086965  16.14330927] km / s
40.09647658080282 km / s
v_inf
21.02773372611768 km / s
max deltav
0.5861195770886941 km / s


60.938921768982425

In [81]:
v_inf = 3.5*u.km/u.s;
qmin = 10*6000*u.km;
2*v_inf*(1+((v_inf)**2*qmin/(Earth.k)).to(u.m/u.m))**-1

<Quantity 2.4613638013139316 km / s>

In [88]:
calc_max_deltav(3.5*u.km/u.s)

<Quantity 2.3647348537123474 km / s>

In [70]:
import math
math.sin(20*math.pi/180)

0.3420201433256687

In [156]:
t_guess_cassini = [15,cassini_js_time.to(u.year).value,cassini_ej_time.to(u.year).value,cassini_ve_time.to(u.year).value]

t_guess_online_traj_tool = [20, 7.93, 2.19,0.15]


t_guess = opt_sol.x

t_guess=t_guess_cassini

t_guess=t_guess_online_traj_tool

bounds = ((10,30),(0.01,10),(0.01,3),(0.01,1))

options = {'maxiter': 60,'disp': True}

def calc_max_deltav(v_inf,qratio=10, body=Earth):
    return 2*v_inf*(1+((v_inf)**2*(qratio*body.R)/(body.k)).to(u.m/u.m))**-1

def saturn_deltav_calculator(tmod):
    print(tmod)
    sol=trajectory_calculator(tmod,plot_on=0,disp_on=0)
    print(sol[3])
    
    
    
    return 5*sol[3][2].to(u.km/u.s).value + sum(sol[3]).to(u.km/u.s).value


def earthflybyConstraint(ts):
    
    sol = trajectory_calculator(t=ts,plot_on=0,disp_on=0)
    
    deltavearth = sol[3][0].to(u.km/u.s)
    print('deltav about earth')
    print(deltavearth)
    v_earth = sol[1][1].v.to(u.km/u.s)
    #print(norm(v_earth))
    v_sat_leaving_earth = sol[2][1].v
    #print(v_sat_leaving_earth)
    #print(norm(v_sat_leaving_earth))
    v_inf = norm(v_sat_leaving_earth-v_earth).to(u.km/u.s)
    #print('v_inf')
    #print(v_inf)
    print('max deltav')
    max_deltav = calc_max_deltav(v_inf,qratio=2, body=Earth)
    print(max_deltav)
    
    return float((max_deltav-deltavearth)/max_deltav)


def jupiterflybyConstraint(ts):
    
    sol = trajectory_calculator(t=ts,plot_on=0,disp_on=0)
    
    deltavjup = sol[3][1].to(u.km/u.s)
    print('deltav about jupiter')
    print(deltavjup)
    v_jup = sol[1][2].v.to(u.km/u.s)
    #print(norm(v_earth))
    v_sat_leaving_jup = sol[2][2].v
    #print(v_sat_leaving_earth)
    #print(norm(v_sat_leaving_earth))
    v_inf = norm(v_sat_leaving_jup-v_jup).to(u.km/u.s)
    #print('v_inf')
    #print(v_inf)
    print('max deltav')
    max_deltav = calc_max_deltav(v_inf,qratio=2, body=Jupiter)
    print(max_deltav)
    
    return float((max_deltav-deltavjup)/max_deltav)


earth_constraint ={'type':'ineq','fun':earthflybyConstraint}
jupiter_constraint ={'type':'ineq','fun':jupiterflybyConstraint}

opt_sol = opt.minimize(saturn_deltav_calculator,x0=t_guess,options=options,bounds=bounds,constraints=(earth_constraint,jupiter_constraint))

opt_sol

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2040-01-01 00:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2032-01-26 13:38:21.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2029-11-17 16:05:57.183. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2029-09-23 21:11:57.182. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

deltav about earth
40.23124326365377 km / s
max deltav
3.4527569455499236 km / s
deltav about jupiter
11.827898181969651 km / s
max deltav
12.746682046029303 km / s
[ 20.     7.93   2.19   0.15]
(<Quantity 40.23124326365377 km / s>, <Quantity 11.827898181969651 km / s>, <Quantity 3.3930164244937746 km / s>)
deltav about earth
40.23124326365377 km / s
max deltav
3.4527569455499236 km / s
deltav about jupiter
11.827898181969651 km / s
max deltav
12.746682046029303 km / s
[ 20.     7.93   2.19   0.15]
(<Quantity 40.23124326365377 km / s>, <Quantity 11.827898181969651 km / s>, <Quantity 3.3930164244937746 km / s>)
[ 20.00000001   7.93         2.19         0.15      ]
(<Quantity 40.23124096362578 km / s>, <Quantity 11.82789817514378 km / s>, <Quantity 3.3930164218556476 km / s>)
[ 20.           7.93000001   2.19         0.15      ]
(<Quantity 40.231245563681725 km / s>, <Quantity 11.827898184568166 km / s>, <Quantity 3.393016416312922 km / s>)
[ 20.           7.93         2.19000001   0.15 

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2044-01-31 13:07:27.495. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2044-01-27 21:27:51.495. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-01-27 03:27:51.870. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2040-01-27 21:27:51.870. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 


deltav about earth
44.8326947008743 km / s
max deltav
1.6247427546306186 km / s
deltav about jupiter
11.67728819254912 km / s
max deltav
12.300851145656576 km / s
[ 20.09789635   7.74013486   2.20941803   0.17037694]
(<Quantity 12.892983813217205 km / s>, <Quantity 10.721941634605555 km / s>, <Quantity 3.497153939662076 km / s>)
deltav about earth
12.892983813217205 km / s
max deltav
4.229995171423662 km / s
deltav about jupiter
10.721941634605555 km / s
max deltav
12.519114179220622 km / s
[ 20.09789635   7.74013486   2.20941803   0.17037694]
(<Quantity 12.892983813217205 km / s>, <Quantity 10.721941634605555 km / s>, <Quantity 3.497153939662076 km / s>)
[ 20.09789637   7.74013486   2.20941803   0.17037694]
(<Quantity 12.892985253206406 km / s>, <Quantity 10.721941549700603 km / s>, <Quantity 3.497153937425106 km / s>)
[ 20.09789635   7.74013487   2.20941803   0.17037694]
(<Quantity 12.892982373228323 km / s>, <Quantity 10.721941714205384 km / s>, <Quantity 3.497153929095171 km / s>)

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2040-02-05 18:10:43.522. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2032-05-10 16:09:23.785. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2030-02-23 16:23:53.494. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2029-12-23 10:52:26.141. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

...


deltav about earth
10.42047618021934 km / s
max deltav
4.866924083268215 km / s
deltav about jupiter
9.99233526304546 km / s
max deltav
12.10251837439254 km / s
[ 20.85841183   8.41062475   2.34498252   0.20704374]
(<Quantity 10.308521250473898 km / s>, <Quantity 10.032180311513107 km / s>, <Quantity 3.02247860496238 km / s>)
deltav about earth
10.308521250473898 km / s
max deltav
4.846084621843399 km / s
deltav about jupiter
10.032180311513107 km / s
max deltav
12.112088178215737 km / s
[ 20.85757084   8.41050952   2.34406487   0.20883505]
(<Quantity 10.30336407333923 km / s>, <Quantity 10.036180143333652 km / s>, <Quantity 3.0226960948222463 km / s>)
deltav about earth
10.30336407333923 km / s
max deltav
4.8439556872868055 km / s
deltav about jupiter
10.036180143333652 km / s
max deltav
12.113044881037359 km / s
[ 20.85748674   8.410498     2.3439731    0.20901418]
(<Quantity 10.302904051454824 km / s>, <Quantity 10.036580279247248 km / s>, <Quantity 3.022717844864485 km / s>)
delta

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2029-11-22 18:50:09.668. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2040-11-09 04:40:27.368. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2032-06-12 06:15:32.040. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2030-02-07 03:04:15.985. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

(<Quantity 10.302854063011639 km / s>, <Quantity 10.03662429589237 km / s>, <Quantity 3.0227202373808755 km / s>)
deltav about earth
10.302854063011639 km / s
max deltav
4.843718880357908 km / s
deltav about jupiter
10.03662429589237 km / s
max deltav
12.113151071948796 km / s
[ 20.85747744   8.41049673   2.34396296   0.20903398]
(<Quantity 10.302853805796685 km / s>, <Quantity 10.036624522659015 km / s>, <Quantity 3.0227202497067003 km / s>)
deltav about earth
10.302853805796685 km / s
max deltav
4.843718759433275 km / s
deltav about jupiter
10.036624522659015 km / s
max deltav
12.113151126163475 km / s
[ 20.85747744   8.41049673   2.34396296   0.20903398]
(<Quantity 10.302853805796685 km / s>, <Quantity 10.036624522659015 km / s>, <Quantity 3.0227202497067003 km / s>)
[ 20.85747746   8.41049673   2.34396296   0.20903398]
(<Quantity 10.302853500402799 km / s>, <Quantity 10.036624447636337 km / s>, <Quantity 3.0227202464477845 km / s>)
[ 20.85747744   8.41049674   2.34396296   0.209033

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2040-11-11 16:50:17.734. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2032-06-09 21:38:12.484. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2030-02-07 01:31:07.527. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2029-11-22 20:05:39.831. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

     fun: 38.47579956118265
     jac: array([ -2.68417115e+01,   2.36149879e+01,   2.07974672e+01,
        -6.72340393e-05])
 message: 'Iteration limit exceeded'
    nfev: 667
     nit: 61
    njev: 60
  status: 9
 success: False
       x: array([ 20.85748431,   8.41051005,   2.34395668,   0.20903364])

In [157]:
sol = trajectory_calculator(t=opt_sol.x,plot_on=0,disp_on=1)

computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  23.362225229178584 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2040-11-09 04:43:35.943. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2032-06-12 06:11:44.023. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2030-02-07 03:04:16.789. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2029-11-22 18:40:56.684. Use Time(..., scale='tdb') instead.



In [158]:
opt_sol.x

array([ 20.85748431,   8.41051005,   2.34395668,   0.20903364])

In [159]:
#sol = trajectory_calculator(t=opt_sol.x,plot_on=0,disp_on=1)

print(opt_sol)

sol = trajectory_calculator([ 21.82450945,   9.33104326,   2.39418937,   0.2])

#print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]


trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_ej/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')

     fun: 38.47579956118265
     jac: array([ -2.68417115e+01,   2.36149879e+01,   2.07974672e+01,
        -6.72340393e-05])
 message: 'Iteration limit exceeded'
    nfev: 667
     nit: 61
    njev: 60
  status: 9
 success: False
       x: array([ 20.85748431,   8.41051005,   2.34395668,   0.20903364])

Total deltav:  22.93831449240922 km / s
Delta vs:  [10.44748454346834, 9.734532392875385, 2.756297556065494]

Times:
Venus:    2029-11-24 17:03:38.175
Earth:    2030-02-05 18:15:38.175
Jupiter:  2032-06-29 05:43:28.638
Saturn:   2041-10-28 09:38:59.419
**
V-E:  0.19999999999999998 yr
E-J:  2.39418937 yr
E-S:  9.33104326 yr



/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-10-28 09:40:08.602. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2032-06-29 05:44:37.822. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2030-02-05 18:16:47.360. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2029-11-24 17:04:47.358. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coord

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [160]:
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()


Total deltav:  22.93831449240922 km / s
Delta vs:  [10.44748454346834, 9.734532392875385, 2.756297556065494]



In [161]:
earthflybyConstraint(opt_sol.x)
print('jup')
jupiterflybyConstraint(opt_sol.x)

deltav about earth
10.302853290332902 km / s
max deltav
4.843718517182257 km / s
jup
deltav about jupiter
10.036657072444871 km / s
max deltav
12.11315615018088 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2040-11-09 04:43:35.943. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2032-06-12 06:11:44.023. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2030-02-07 03:04:16.789. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2029-11-22 18:40:56.684. Use Time(..., scale='tdb') instead.



0.17142510605751587